In [3]:
from Stitcher import stitch_neuron
import datajoint as dj
import numpy as np
import datajoint as dj
import trimesh
import time

In [4]:
# testing which table to pull from
pinky = dj.create_virtual_module("pinky","microns_pinky")
schema = dj.schema("microns_pinky")

Connecting celiib@10.28.0.34:3306


In [5]:
pinky.ExcitatoryStitchedMeshVp2()# & "stitched_addon_perc>1.0" #& "unstitched_perc>0.001"

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_vertices number of vertices in this mesh,n_triangles number of triangles in this mesh,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices),n_pieces number of unconnected mesh pieces outside the largest mesh piece,largest_piece_perc number of faces percentage of largest mesh piece in respect to total mesh,outside_perc number of faces percentage of mesh outside the biggest mesh piece,n_stitched number of mesh pieces stitched back to main mesh,stitched_addon_perc number of faces percentage of pieces that were stitched back in respect to largest mesh piece,n_unstitched number of mesh pieces remaining unstitched back to main mesh,unstitched_perc number of faces percentage of pieces that were not in respect to largest mesh piece
3,648518346341371119,639722,1274428,=BLOB=,=BLOB=,0,0.96814,0.96814,0,0.00000,0,0.00000
3,648518346349386137,81151,161100,=BLOB=,=BLOB=,0,0.96487,0.96487,0,0.00000,0,0.00000
3,648518346349470171,1102842,2198222,=BLOB=,=BLOB=,43,0.73927,0.88110,42,0.19123,1,0.00002
3,648518346349471156,783999,1565239,=BLOB=,=BLOB=,16,0.90102,0.90916,16,0.00908,0,0.00000
3,648518346349471500,743900,1489208,=BLOB=,=BLOB=,2,0.93106,0.93124,2,0.00019,0,0.00000
3,648518346349471562,1294767,2579165,=BLOB=,=BLOB=,18,0.96604,0.97157,18,0.00574,0,0.00000
3,648518346349471565,2183207,4352068,=BLOB=,=BLOB=,16,0.92285,0.98378,14,0.06602,2,0.00002


In [ ]:
pinky.PymeshfixDecimatedExcitatoryStitchedMesh

# get the stats on the stitching

In [7]:
stitched_addon_perc,unstitch_perc = pinky.ExcitatoryStitchedMeshVp2.fetch("stitched_addon_perc","unstitched_perc")

In [9]:
np.mean(stitched_addon_perc)

Decimal('0.3401833024691358024691358025')

In [17]:
sorted_unstitched = np.sort(unstitch_perc)
len(np.where(sorted_unstitched > 0.001)[0])

17

In [21]:
np.where(unstitch_perc == np.max(unstitch_perc))[0][0]

107

In [23]:
new_unstitched = np.delete(unstitch_perc,107)

In [25]:
np.where(new_unstitched > 0.001)

(array([ 61,  78,  88, 103, 117, 132, 142, 175, 241, 249, 251, 292, 308,
        311, 321, 322]),)

In [29]:
print("max percentage unstitched = " + str(np.max(new_unstitched)))
print("Number of neurons with greater than 0.001 unstitched = " + str(len(np.where(new_unstitched > 0.01)[0])))
print("Number of neurons with greater than 0.001 unstitched = " + str(len(np.where(new_unstitched > 0.005)[0])))
print("Number of neurons with greater than 0.001 unstitched = " + str(len(np.where(new_unstitched > 0.001)[0])))



max percentage unstitched = 0.01749
Number of neurons with greater than 0.001 unstitched = 1
Number of neurons with greater than 0.001 unstitched = 2
Number of neurons with greater than 0.001 unstitched = 16


In [21]:
# get the original mesh
segment_id = 648518346349482676 #1st tested mesh
segment_id = 648518346349493487
key = dict(segmentation=3,segment_id = segment_id)
verts,faces = (pinky.Mesh() & key).fetch1("vertices","triangles")

full_mesh = trimesh.Trimesh()
full_mesh.vertices = verts
full_mesh.faces = faces

from print_trimesh import print_trimesh
print_trimesh(full_mesh,"./neurons/" + str(segment_id) +"_full.off")




In [35]:
# get the newly stitched mesh
# get the original mesh
segment_id = 648518346349482020
key = dict(segmentation=3,segment_id = segment_id)
verts,faces = (pinky.ExcitatoryStitchedMeshVp2() & key).fetch1("vertices","triangles")

stitched_mesh = trimesh.Trimesh()
stitched_mesh.vertices = verts
stitched_mesh.faces = faces

from print_trimesh import print_trimesh
print_trimesh(stitched_mesh,"./neurons/" + str(segment_id) +"_stitched_vp2.off")


In [22]:
#get the largest piece of the non-stitched mesh
# get the original mesh
significance_threshold = 0
n_sample_points=3
segment_id = 648518346349493487
key = dict(segmentation=3,segment_id = segment_id)
verts,faces = (pinky.Mesh() & key).fetch1("vertices","triangles")

unfiltered_mesh = trimesh.Trimesh()
unfiltered_mesh.vertices = verts
unfiltered_mesh.faces = faces


mesh_pieces = unfiltered_mesh.split(only_watertight=False)

print(f"There were {len(mesh_pieces)} pieces after mesh split")

significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

print(f"There were {len(significant_pieces)} pieces found after size threshold")
if len(significant_pieces) <=0:
    print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
    
else:
    #find piece with largest size
    max_index = 0
    max_face_len = len(significant_pieces[max_index].faces)

    for i in range(1,len(significant_pieces)):
        if max_face_len < len(significant_pieces[i].faces):
            max_index = i
            max_face_len = len(significant_pieces[i].faces)

    print("max_index = " + str(max_index))
    print("max_face_len = " + str(max_face_len))

    final_mesh_pieces = []

    main_mesh = significant_pieces[max_index]
    
    #final_mesh_pieces.append(main_mesh)
    for i,mesh in enumerate(significant_pieces):
        if i != max_index:
            #get a random sample of points
            # points = np.array(mesh.vertices[:n_sample_points,:]) # OLD WAY OF DOING THIS
            idx = np.random.randint(len(mesh.vertices), size=n_sample_points)
            points = mesh.vertices[idx,:]
            
            
            start_time = time.time()
            signed_distance = trimesh.proximity.signed_distance(main_mesh,points)
            #print(f"Total time = {time.time() - start_time}")

            outside_percentage = sum(signed_distance < 0)/n_sample_points
            if outside_percentage > 0.9:
                final_mesh_pieces.append(mesh)
                #print(f"Mesh piece {i} OUTSIDE mesh")
            else:
                #print(f"Mesh piece {i} inside mesh :( ")
                pass
                
    

    from print_trimesh import print_trimesh
    print_trimesh(main_mesh,"./neurons/" + str(segment_id) +"_largest.off")
    
    


There were 434 pieces after mesh split
There were 434 pieces found after size threshold
max_index = 1
max_face_len = 552716


In [13]:
significant_pieces[4].show()

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [ ]:
# get the 

In [ ]:
def filter_mesh_significant_outside_pieces(unfiltered_mesh,significance_threshold=2000,n_sample_points=1000):
    """
    Purpose; will take in a full, unfiltered mesh and find the biggest mesh piece, and then return a list of that mesh 
    with all of the other mesh fragments that are both above the significance_threshold AND outside of the biggest mesh piece

    Pseudocode: 
    1) split the meshes to unconnected pieces
    2) Filter the meshes for only those above the significance_threshold
    3) find the biggest mesh piece
    4) Iterate through all of the remaining pieces:
        a. Determine if mesh inside or outside main mesh
        b. If outside add to final list to return

    Returns: 
    1) list of significant mesh pieces, including the main one that are not inside of main mesh

    """

    mesh_pieces = unfiltered_mesh.split(only_watertight=False)
    
    print(f"There were {len(mesh_pieces)} pieces after mesh split")

    significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

    print(f"There were {len(significant_pieces)} pieces found after size threshold")
    if len(significant_pieces) <=0:
        print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
        return []

    #find piece with largest size
    max_index = 0
    max_face_len = len(significant_pieces[max_index].faces)

    for i in range(1,len(significant_pieces)):
        if max_face_len < len(significant_pieces[i].faces):
            max_index = i
            max_face_len = len(significant_pieces[i].faces)

    print("max_index = " + str(max_index))
    print("max_face_len = " + str(max_face_len))

    final_mesh_pieces = []

    main_mesh = significant_pieces[max_index]

    #final_mesh_pieces.append(main_mesh)
    for i,mesh in enumerate(significant_pieces):
        if i != max_index:
            #get a random sample of points
            # points = np.array(mesh.vertices[:n_sample_points,:]) # OLD WAY OF DOING THIS
            idx = np.random.randint(len(mesh.vertices), size=n_sample_points)
            points = mesh.vertices[idx,:]
            
            
            start_time = time.time()
            signed_distance = trimesh.proximity.signed_distance(main_mesh,points)
            #print(f"Total time = {time.time() - start_time}")

            outside_percentage = sum(signed_distance < 0)/n_sample_points
            if outside_percentage > 0.9:
                final_mesh_pieces.append(mesh)
                print(f"Mesh piece {i} OUTSIDE mesh")
            else:
                print(f"Mesh piece {i} inside mesh :( ")
                pass
                
    return main_mesh,final_mesh_pieces




In [ ]:
main_mesh,final_mesh_pieces = filter_mesh_significant_outside_pieces(current_mesh,
                        significance_threshold=30,
                        n_sample_points=3)


In [ ]:
from print_trimesh import print_trimesh
print_trimesh(main_mesh,"./neurons/" + str(segment_id) +"_check_filtered.off")

In [ ]:
len(final_mesh_pieces) == 0